In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import momepy
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

from shapely.ops import nearest_points, unary_union, cascaded_union

In [2]:
# чтение файлов

In [3]:
graph = gpd.read_file('../lines_to_graph/with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [4]:
nodes = gpd.read_file('../lines_to_graph/with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [5]:
buildings = gpd.read_file('./all_bui.json',encoding='utf-8')
buildings.total_ppl = buildings.total_ppl.astype(float)

In [6]:
stps_big = pd.read_csv('../lines_to_graph/total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [7]:
border_sa = gpd.read_file('./data/outer_border_SA.shp',encoding='utf-8')

In [8]:
# предварительный фильтр

In [9]:
stps_big = stps_big[['stop_id', 'stop_name', 'lat', 'lon']]
stps_big = stps_big.drop_duplicates(subset=['stop_id']).reset_index(drop=True)

In [10]:
lst_geo=[]
lst_lat=list(stps_big.lat)
lst_lon=list(stps_big.lon)
i=0
for i in tqdm(range(len(stps_big))):
    one_pt = Point(lst_lon[i], lst_lat[i])
    lst_geo.append(one_pt)
# 

In [11]:
gdf_stops_all = gpd.GeoDataFrame(stps_big, geometry=lst_geo)
gdf_stops_all.crs='epsg:4326'

In [12]:
gdf_stops = gpd.sjoin(gdf_stops_all, border_sa[['geometry']], how='inner', op='intersects')
gdf_stops = gdf_stops.drop('index_right', axis=1).reset_index(drop=True)

In [13]:
shrt_bui = buildings[['osm_id', 'geometry']].copy()

In [14]:
# убрать жд пути из графа
graph_ped = graph[graph.type_ped != 'no_ped'].reset_index(drop=True)

graph_notdir = graph_ped[graph_ped.direction == 'direct'].reset_index(drop=True)
graph_notdir = graph_notdir[['link_id', 'from_node', 'to_node', 'geometry']]


graph_notdir_32640 = graph_notdir.copy()
graph_notdir_32640 = graph_notdir_32640.to_crs('epsg:32640')
graph_notdir_32640['length'] = graph_notdir_32640.geometry.length
fields = list(graph_notdir_32640.columns)

In [15]:
np_gnd = graph_notdir_32640.to_numpy()
ind_frnd = list(graph_notdir_32640.columns).index('from_node')
ind_tond = list(graph_notdir_32640.columns).index('to_node')
ind_geo = list(graph_notdir_32640.columns).index('geometry')
ind_li = list(graph_notdir_32640.columns).index('link_id')


In [16]:
# Алгоритм группирования зданий
# 0 удалить нежилые по тегу - проверить если нет total_ppl
# 1 total_ppl > 20 - многоквартирный
# Если люди есть и их меньше 20 - то ИЖС.
# Если людей нет, но этажей больше 4 - то многоквартирный .
# 2 нет жителей - если этажей (bui_lvl) меньше 4 - ИЖС
# 3 нет этажей, нет людей, площадь полигона (чисто полигона) 
# если меньше 400 - ИЖС, если больше - многоквартирный


In [18]:
# Если людей больше 20 - многоквартирные
# Если меньше (или их нет):
#     Если больше 4 этажей - многоквартирные
#     Если меньше:
#         Если площадь больше 400 - многоквартирные
#         Если меньше - ИЖС

In [19]:
bui_liv = buildings[~(buildings.total_ppl.isna() 
                     & (buildings.bui_group == 'нежилое'))].reset_index(drop=True)

In [20]:
bui_liv_32640 = bui_liv.copy()
bui_liv_32640 = bui_liv_32640.to_crs('epsg:32640')

In [21]:
np_bl = bui_liv_32640.to_numpy()
ind_grp = list(bui_liv_32640.columns).index('bui_group')
ind_livsq = list(bui_liv_32640.columns).index('living_rooms_sq')
ind_ppl = list(bui_liv_32640.columns).index('total_ppl')
ind_blvl = list(bui_liv_32640.columns).index('bui_lvl')
ind_geo = list(bui_liv_32640.columns).index('geometry')

In [22]:
lst_liv_type = []
lst_new_grp=[]

for i in tqdm(range(len(np_bl))):
    bui_group = np_bl[i, ind_grp]
    liv_sq = np_bl[i, ind_livsq]
    ttl_ppl = np_bl[i, ind_ppl]
    bui_lvl = np_bl[i, ind_blvl]
    geo = np_bl[i, ind_geo]
    
    if ((bui_group == 'нежилое') & (ttl_ppl == ttl_ppl) & (liv_sq == liv_sq)):
        new_grp = 'жилое'
    else:
        new_grp = bui_group
    #
    lst_new_grp.append(new_grp)
    
    #
    if ttl_ppl > 20:
        liv_type = 'многоквартирный'
    else:
        if bui_lvl > 4:
            liv_type = 'многоквартирный'
        else:
            if geo.area > 400:
                liv_type = 'многоквартирный'
            else:
                liv_type = 'ИЖС'
    #
    lst_liv_type.append(liv_type)
# 

In [23]:
bui_liv['bui_group'] = lst_new_grp
bui_liv['bui_liv_type'] = lst_liv_type

In [24]:
bui_liv_400 = bui_liv[bui_liv.bui_liv_type == 'многоквартирный'].reset_index(drop=True)
bui_liv_1350 = bui_liv[bui_liv.bui_liv_type == 'ИЖС'].reset_index(drop=True)

In [25]:
bui_liv_400_shrt = bui_liv_400[['osm_id', 'bui_liv_type', 'geometry']]

In [26]:
bui_liv_1350_shrt = bui_liv_1350[['osm_id', 'bui_liv_type', 'geometry']]

In [27]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []

    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c

    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    return result
########

In [28]:
def find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff):
    uu_stops = gdf_stops.geometry.unary_union
    centr_bui = gdf_one_bui.geometry[0].centroid
    nrst_pt = nearest_points(centr_bui, uu_stops)[1]
    
    nearest_stop_id = list(gdf_stops[gdf_stops.geometry == nrst_pt]['stop_id'])[0]
    nearest_stop_geo = list(gdf_stops[gdf_stops.geometry == nrst_pt]['geometry'])[0]
    if gdf_buff.geometry[0].intersects(nearest_stop_geo):
        return [nearest_stop_id, nearest_stop_geo]
    else:
        return []

In [29]:
def get_closest_node(gdf_obj, linkid, graph_in_buff):
    line_geo = list(graph_in_buff[graph_in_buff.link_id == linkid]['geometry'])[0]
    start = Point(line_geo.coords[0])
    end = Point(line_geo.coords[-1])
    obj_centr = gdf_obj['geometry'][0]#.centroid

    if start.distance(obj_centr) < end.distance(obj_centr):
        return start
    else:
        return end
# 

In [30]:
def get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped):

    try:
        nearest_stop = find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff)
    except:
        nearest_stop = []
    #
    if nearest_stop != []:
        gdf_near_stop = gpd.GeoDataFrame(data=[nearest_stop], columns=['stop_id', 'geometry'])
        gdf_near_stop.crs=graph_ped.crs
        graph_in_buff = intersect_using_spatial_index(graph_ped, gdf_buff[['geometry']])
        centr_bui = gdf_one_bui.copy()
        centr_bui.geometry = centr_bui.geometry.centroid

        text_trap = io.StringIO()
        with redirect_stdout(text_trap):
            closest_link_bui = mm.get_network_id(gdf_one_bui, graph_in_buff, 'link_id')[0]
            closest_link_stop = mm.get_network_id(gdf_near_stop, graph_in_buff, 'link_id')[0]

        closest_node_bui = get_closest_node(gdf_one_bui, closest_link_bui, graph_in_buff)
        closest_node_stop = get_closest_node(gdf_near_stop, closest_link_stop, graph_in_buff)

        one_G = mm.gdf_to_nx(graph_in_buff)
        res_lst = nx.shortest_path(one_G, closest_node_bui.coords[0], closest_node_stop.coords[0])

        if len(res_lst) == 1:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = 0
        elif len(res_lst) == 0:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = -1
        else:
            line_path = LineString(res_lst)
            path_lngth = 1
    # 
        return [nearest_stop[0], path_lngth, res_lst, line_path]
    else:
        return []

In [31]:
def generate_graph(edges, fields):
    G = nx.Graph()

    key = 0
    for index, row in edges.iterrows():
        first = row.from_node
        last = row.to_node

        data = [row[f] for f in fields]
        attributes = dict(zip(fields, data))
        G.add_edge(first, last, key=key, **attributes)
        key += 1
    # 
    return G

In [32]:
def get_lst_end_nodes(egg):
    lst_nodes_end=[]
    lst_nodes = list(egg.nodes())
    i=0
    for i in (range(len(lst_nodes))):
        one_node = lst_nodes[i]
        lst_neigh = list(nx.all_neighbors(egg, one_node))
        if len(lst_neigh) < 2:
            lst_nodes_end.append(one_node)
    #
    return lst_nodes_end

In [33]:
def get_strt_end_paths(egg, closest_node, lst_nodes_end):
    
    shortest_paths = list(nx.all_pairs_dijkstra_path(egg, weight='length'))
    
    start_ind=0
    i=0
    for i in range(len(shortest_paths)):
        if shortest_paths[i][0] == closest_node:
            start_ind = i
            break
    # 
    lst_paths_to_ends = []
    i=0
    for nd in shortest_paths[start_ind][1]:
        if nd in lst_nodes_end:
            lst_paths_to_ends.append(shortest_paths[start_ind][1][nd])
    # 
    try:
        lst_paths_to_ends.remove([closest_node])
    except:
        pass
    return lst_paths_to_ends

In [34]:
def find_closest_node(one_stop_gdf, nodes_in_buff):

    stop_pt = one_stop_gdf.geometry[0]
    np_nib = nodes_in_buff.to_numpy()
    ind_ndid = list(nodes_in_buff.columns).index('nodeID')
    ind_geo = list(nodes_in_buff.columns).index('geometry')

    closest_node = np_nib[0, ind_ndid]
    dstnc = 100000000
    i=0
    for i in range(len(np_nib)):
        node_pt = np_nib[i, ind_geo]
        if stop_pt.distance(node_pt) < dstnc:
            dstnc = stop_pt.distance(node_pt)
            closest_node = np_nib[i, ind_ndid]
    # 

    return closest_node

In [35]:
def get_one_stop_graph(one_stop_gdf, nodes, G, buff):
    
    buff_big = buff + 10
    gdf_buff = one_stop_gdf.copy()
    gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32640').buffer(buff_big).to_crs('epsg:4326')
    nodes_in_buff = intersect_using_spatial_index(nodes, gdf_buff[['geometry']]).reset_index(drop=True)

    closest_node = find_closest_node(one_stop_gdf, nodes_in_buff)
    lst_nodes_buff = list(nodes_in_buff['nodeID'])
    lst_paths_to_ends = path_to_ends(lst_nodes_buff, G, closest_node)

#     egg = nx.ego_graph(G, closest_node, radius=400, distance='length')
#     edglst = nx.to_pandas_edgelist(egg)
#     lst_nodes_end = get_lst_end_nodes(egg)
#     lst_paths_to_ends = get_strt_end_paths(egg, closest_node, lst_nodes_end)
    return lst_paths_to_ends

In [36]:
def path_to_ends(lst_nodes_buff, G, closest_node):
    lst_done = []
    lst_all=[]
    k=0
    for k in range(len(lst_nodes_buff)):
        one_node = lst_nodes_buff[k]
        if one_node not in lst_done:
            lst_done.append(one_node)
            lngth = nx.dijkstra_path_length(G, closest_node, one_node, weight='length')
            if lngth < 400:
                lst_shtpth = list(nx.dijkstra_path(G, closest_node, one_node, weight='length'))
                lst_all.append(lst_shtpth)
                j=0
                for j in range(len(lst_shtpth)):
                    if lst_shtpth[j] not in lst_done:
                        lst_done.append(lst_shtpth[j])
    # 
    return lst_all

In [37]:
def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]

In [38]:
def get_one_line(lst_one, np_gnd, ind_frnd, ind_tond, ind_geo):
    
    lst_lines=[]
    i=0
    for i in range(1, len(lst_one)):
        try:
            row = np_gnd[((np_gnd[:,ind_frnd] == lst_one[i-1]) 
                            & (np_gnd[:,ind_tond] == lst_one[i]))][0]
        except:
            row = np_gnd[((np_gnd[:,ind_tond] == lst_one[i-1]) 
                            & (np_gnd[:,ind_frnd] == lst_one[i]))][0]
        #
        geo = row[ind_geo]
        lst_lines.append(geo)
    #
    multi_line = MultiLineString(lst_lines)
    lns = shapely.ops.linemerge(multi_line)
    
    return lns

In [39]:
def get_leftovers(np_gnd, ind_frnd, ind_tond, ind_geo, last_id, lns, lst_one, lng_cut):
    left_lines = np_gnd[(
                    ((np_gnd[:,ind_tond] == last_id) & ~(np.isin(np_gnd[:,ind_frnd], lst_one)) 
                     | ((np_gnd[:,ind_frnd] == last_id) & ~(np.isin(np_gnd[:,ind_tond], lst_one))))
                )]
    #

    left_length = lng_cut - lns.length
    lst_leftovers=[]
    i=0
    for i in range(len(left_lines)):
        one_line_left = left_lines[i][ind_geo]
        left_length = lng_cut - lns.length
        if one_line_left.length > left_length:
            if ((one_line_left.coords[0] != lns.coords[-1]) 
                & (one_line_left.coords[-1] != lns.coords[0])):
                one_line_left2 = one_line_left.coords[:]
                one_line_left2.reverse()
                one_line_left = LineString(one_line_left2)
            #
            left_length = one_line_left.length - left_length
            lst_cut_left = cut(one_line_left, left_length)
            if lst_cut_left == []:
                lst_cut_left = [one_line_left]
            if ((lst_cut_left[0].coords[0] == lns.coords[-1]) 
                | (lst_cut_left[0].coords[-1] == lns.coords[0])):
                cut_left = lst_cut_left[0]
            else:
                try:
                    cut_left = lst_cut_left[1]
                except:
                    cut_left = lst_cut_left[0]
            #
        else:
            cut_left = one_line_left
        lst_leftovers.append(cut_left)
    # 

    return lst_leftovers

In [40]:
def check_leftovers(np_gnd, ind_frnd, ind_tond, ind_geo, lns2, lst_one2, lng_cut):
    
    last_id = lst_one2[-1]
    
    if lns2.length > lng_cut:
        try:
            cutted_line = cut(lns2, lng_cut)[0]
        except:
            cutted_line = lns2
        lst_leftovers=[]
    elif lns2.length == lng_cut:
        cutted_line = lns
        lst_leftovers=[]
    else:
        cutted_line = lns2
        lst_leftovers = get_leftovers(np_gnd, ind_frnd, ind_tond, ind_geo, last_id, lns2, lst_one2, lng_cut)
    #
    return cutted_line, lst_leftovers
#

In [41]:
def get_one_end_lines(lst_one, np_gnd, ind_frnd, ind_tond, ind_geo, buff):

    lng_cut = buff

    lns = get_one_line(lst_one, np_gnd, ind_frnd, ind_tond, ind_geo)

    cutted_line, lst_leftovers = check_leftovers(np_gnd, ind_frnd, ind_tond, ind_geo, lns, lst_one, lng_cut)

    lst_lines_one_end = [cutted_line] + lst_leftovers
    
    l1=[]
    i=0
    for i in range(len(lst_one)):
        one_link_lst = lst_one[0:i+1]

        if len(one_link_lst) > 1:
            one_line = get_one_line(one_link_lst, np_gnd, ind_frnd, ind_tond, ind_geo)
            l1.append(one_line)
            cutted_line_sm, lst_leftovers_sm = check_leftovers(np_gnd, ind_frnd, ind_tond, ind_geo, one_line, one_link_lst, lng_cut)
            if lst_leftovers_sm != lst_leftovers:
                j=0
                for j in range(len(lst_leftovers_sm)):
                    if lst_leftovers_sm[j] not in lst_lines_one_end:
                        lst_lines_one_end.append(lst_leftovers_sm[j])
    #

    return lst_lines_one_end

In [42]:
def get_one_stop_sprut(lst_paths_to_ends, np_gnd, ind_frnd, ind_tond, ind_geo, buff):
    lst_all_ends=[]
    i=0
    for i in range(len(lst_paths_to_ends)):
        lst_one = lst_paths_to_ends[i]
        lst_lines_one_end = get_one_end_lines(lst_one, np_gnd, ind_frnd, ind_tond, ind_geo, buff)
        for k in range(len(lst_lines_one_end)):
            if lst_lines_one_end[k] not in lst_all_ends:
                lst_all_ends.append(lst_lines_one_end[k])
    #
    return lst_all_ends

In [74]:
def get_sj_and_spruts(lngth, graph_notdir_32640, G, gdf_stops, shrt_bui, buff_big, buff_small):
    
    np_gnd = graph_notdir_32640.to_numpy()
    ind_frnd = list(graph_notdir_32640.columns).index('from_node')
    ind_tond = list(graph_notdir_32640.columns).index('to_node')
    ind_geo = list(graph_notdir_32640.columns).index('geometry')
    ind_li = list(graph_notdir_32640.columns).index('link_id')

    lst_cu_spruts = []
    lst_stop_id = []
    
    i=0
    for i in tqdm(range(lngth)):
    # for i in tqdm(range(len(gdf_stops))):
        one_stop_gdf = gdf_stops.iloc[[i]].reset_index(drop=True)
        stop_id = one_stop_gdf.stop_id[0]
        lst_paths_to_ends = get_one_stop_graph(one_stop_gdf, nodes, G, buff_big)
        sprut_ml_one_stop = get_one_stop_sprut(lst_paths_to_ends, np_gnd, 
                                               ind_frnd, ind_tond, ind_geo, buff_big)
        cu_sprut = cascaded_union(sprut_ml_one_stop)
        lst_cu_spruts.append(cu_sprut)
        lst_stop_id.append(stop_id)
    # 

    spruts = gpd.GeoDataFrame(data=lst_stop_id,geometry=list(lst_cu_spruts), columns=['stop_id'])
    spruts['ped_avail_m'] = buff_big
    spruts.crs = 'epsg:32640'

    spruts_buff = spruts.copy()
    spruts_buff.geometry = spruts_buff.geometry.buffer(buff_small)
    
    spruts_buff = spruts_buff.to_crs('epsg:4326')
    spruts = spruts.to_crs('epsg:4326')
    
    print(datetime.now())
    if buff_big == 400:
        tm = (len(lst_cu_spruts) * 6) / 60
    else:
        tm = (len(lst_cu_spruts) * 10) / 60
    print("time left to sjoin spruts_buff with buildings, min:", tm)

    sj = gpd.sjoin(shrt_bui, spruts_buff, how='inner', 
                   op='intersects').drop("index_right", axis=1).reset_index(drop=True)
    sj = sj.drop_duplicates(subset=['osm_id']).reset_index(drop=True)
    print(datetime.now())
    
    return sj, spruts_buff, spruts

In [44]:
# https://github.com/martinfleis/momepy/blob/v0.3.0/momepy/utils.py#L117-L156
%time G = generate_graph(graph_notdir_32640, fields)

Wall time: 21.4 s


In [75]:
shrt_bui = bui_liv_400_shrt.copy()
buff_big, buff_small = 400, 100
# lngth = len(gdf_stops)
lngth = 3
%time sj_400, spruts_buff_400, spruts_400 = get_sj_and_spruts(lngth, graph_notdir_32640, G, gdf_stops, shrt_bui, buff_big, buff_small)


2020-09-14 09:10:13.896278
time left to sjoin spruts_buff with buildings, min: 0.3
2020-09-14 09:10:16.256536
Wall time: 28.7 s


In [71]:
shrt_bui = bui_liv_1350_shrt.copy()
buff_big, buff_small = 1350, 150
# lngth = len(gdf_stops)
lngth = 3
%time sj_1350, spruts_buff_1350, spruts_1350 = get_sj_and_spruts(graph_notdir_32640, G, gdf_stops, shrt_bui, buff_big, buff_small)


2020-09-14 08:55:37.907041
time left to sjoin spruts_buff with buildings, min: 0.3
2020-09-14 08:55:51.136320
Wall time: 44.3 s


In [67]:
num = 4
spruts_buff_1350.to_file('spruts_buff_1350_{}.json'.format(num), driver='GeoJSON', 
                         encoding='utf-8')
spruts_1350.to_file('spruts_1350_{}.json'.format(num), driver='GeoJSON', encoding='utf-8')
sj_1350.to_file('sj_1350_{}.json'.format(num), driver='GeoJSON', encoding='utf-8')

In [65]:
num = 4
spruts_buff_400.to_file('spruts_buff_400_{}.json'.format(num), driver='GeoJSON')
spruts_400.to_file('spruts_400_{}.json'.format(num), driver='GeoJSON')
sj_400.to_file('sj_400_{}.json'.format(num), driver='GeoJSON')

In [ ]:
# прилепить оставшиеся здания (вне буффера)

In [83]:
new_bui_liv = bui_liv[(~(bui_liv.osm_id.isin(sj_400.osm_id)) 
                        & (~bui_liv.osm_id.isin(sj_1350.osm_id)))].reset_index(drop=True)
new_bui_liv['stop_id'] = None
new_bui_liv['ped_avail_m'] = None
new_bui_liv = new_bui_liv[['osm_id', 'bui_liv_type', 'geometry', 'stop_id', 'ped_avail_m']]

In [87]:
total_bui_liv = sj_400.append(sj_1350).append(new_bui_liv).reset_index(drop=True)

In [88]:
total_bui_liv.head(2)

,osm_id,bui_liv_type,geometry,stop_id,ped_avail_m
0,40532043,многоквартирный,"POLYGON ((50.17355 53.19140, 50.17246 53.19137...",stop__10000931,400
1,41601932,многоквартирный,"POLYGON ((50.17797 53.19571, 50.17818 53.19571...",stop__10000931,400


In [ ]:
num = 4
total_bui_liv.to_file('total_bui_liv_{}.json'.format(num), driver='GeoJSON', 
                         encoding='utf-8')